<a href="https://colab.research.google.com/github/Jobby-John/Week-6-ip/blob/main/John_chege_ip_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [301]:
#Load all necessary libraries & prerequisites

import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sp

from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [302]:
rank= pd.read_csv('fifa_ranking.csv')
result= pd.read_csv('results.csv')

In [303]:
rank.head(10)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
5,6,Republic of Ireland,IRL,0.0,54,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
6,7,Russia,RUS,0.0,52,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
7,8,Brazil,BRA,0.0,55,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
8,9,Norway,NOR,0.0,49,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
9,10,Denmark,DEN,0.0,51,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08


In [304]:
result.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,False
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,False


In [305]:
#Check columns in both tables to better assess missing parts in each data set, and merging possibilities
print(result.columns)
print(rank.columns)

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral'],
      dtype='object')
Index(['rank', 'country_full', 'country_abrv', 'total_points',
       'previous_points', 'rank_change', 'cur_year_avg',
       'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
       'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg',
       'three_year_ago_weighted', 'confederation', 'rank_date'],
      dtype='object')


EDA

In [306]:
#Merging the 2 datasets

Fifa = result.merge(rank, left_on = ['date', 'home_team'],
                        right_on = ['rank_date', 'country_full'], how = 'inner')
Fifa.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,59,Bolivia,BOL,0.0,13,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
1,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False,8,Brazil,BRA,0.0,55,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
2,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False,35,Ecuador,ECU,0.0,23,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
3,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False,65,Guinea,GUI,0.0,23,-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1993-08-08
4,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,67,Paraguay,PAR,0.0,22,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [307]:
#drop the unnecessary columns 
Fifa.drop(columns = ['country_abrv','tournament','total_points', 'previous_points', 
                         'rank_change', 'cur_year_avg','cur_year_avg_weighted', 'last_year_avg',
                         'last_year_avg_weighted', 'two_year_ago_avg', 'two_year_ago_weighted', 
                         'three_year_ago_avg', 'three_year_ago_weighted'], axis = 1, inplace = True)

In [308]:
Fifa.head(5)

,date,home_team,away_team,home_score,away_score,city,country,neutral,rank,country_full,confederation,rank_date
0,1993-08-08,Bolivia,Uruguay,3,1,La Paz,Bolivia,False,59,Bolivia,CONMEBOL,1993-08-08
1,1993-08-08,Brazil,Mexico,1,1,Maceió,Brazil,False,8,Brazil,CONMEBOL,1993-08-08
2,1993-08-08,Ecuador,Venezuela,5,0,Quito,Ecuador,False,35,Ecuador,CONMEBOL,1993-08-08
3,1993-08-08,Guinea,Sierra Leone,1,0,Conakry,Guinea,False,65,Guinea,CAF,1993-08-08
4,1993-08-08,Paraguay,Argentina,1,3,Asunción,Paraguay,False,67,Paraguay,CONMEBOL,1993-08-08


In [309]:
# Rename rank column to home_team_rank
Fifa.rename(columns = {'rank' : 'home_team_rank'}, inplace = True)
print(Fifa.columns)

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score', 'city',
       'country', 'neutral', 'home_team_rank', 'country_full', 'confederation',
       'rank_date'],
      dtype='object')


In [310]:
Fifa.head(5)

,date,home_team,away_team,home_score,away_score,city,country,neutral,home_team_rank,country_full,confederation,rank_date
0,1993-08-08,Bolivia,Uruguay,3,1,La Paz,Bolivia,False,59,Bolivia,CONMEBOL,1993-08-08
1,1993-08-08,Brazil,Mexico,1,1,Maceió,Brazil,False,8,Brazil,CONMEBOL,1993-08-08
2,1993-08-08,Ecuador,Venezuela,5,0,Quito,Ecuador,False,35,Ecuador,CONMEBOL,1993-08-08
3,1993-08-08,Guinea,Sierra Leone,1,0,Conakry,Guinea,False,65,Guinea,CAF,1993-08-08
4,1993-08-08,Paraguay,Argentina,1,3,Asunción,Paraguay,False,67,Paraguay,CONMEBOL,1993-08-08


In [311]:
Fifa.drop(columns = ['date','city', 'rank_date', 'confederation'], axis = 1, inplace = True)

In [312]:
Fifa.head(5)

,home_team,away_team,home_score,away_score,country,neutral,home_team_rank,country_full
0,Bolivia,Uruguay,3,1,Bolivia,False,59,Bolivia
1,Brazil,Mexico,1,1,Brazil,False,8,Brazil
2,Ecuador,Venezuela,5,0,Ecuador,False,35,Ecuador
3,Guinea,Sierra Leone,1,0,Guinea,False,65,Guinea
4,Paraguay,Argentina,1,3,Paraguay,False,67,Paraguay


In [313]:
# Check for null values and drop them if necessary
Fifa.isnull().any()
#There are no null values and our data seems to be clean

home_team         False
away_team         False
home_score        False
away_score        False
country           False
neutral           False
home_team_rank    False
country_full      False
dtype: bool

Feature engineering
    
    
    Create home team rank, and away team rank columns
    Turn the neutral column into data
    Get a win column also 


In [314]:
Fifa = Fifa.merge(rank, left_on = ['away_team'],
                        right_on = ['country_full'], how = 'inner')

In [315]:
Fifa.head(3)

,home_team,away_team,home_score,away_score,country,neutral,home_team_rank,country_full_x,rank,country_full_y,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,Bolivia,Uruguay,3,1,Bolivia,False,59,Bolivia,22,Uruguay,URU,0.0,47,-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
1,Bolivia,Uruguay,3,1,Bolivia,False,59,Bolivia,15,Uruguay,URU,0.0,45,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-09-23
2,Bolivia,Uruguay,3,1,Bolivia,False,59,Bolivia,17,Uruguay,URU,0.0,51,-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-10-22


In [316]:
#Now we rename the new rank column to away_team_rank, and drop the unnecessary columns again. 
Fifa.rename(columns = {'rank' : 'away_team_rank'}, inplace = True)
print(Fifa.columns)

Index(['home_team', 'away_team', 'home_score', 'away_score', 'country',
       'neutral', 'home_team_rank', 'country_full_x', 'away_team_rank',
       'country_full_y', 'country_abrv', 'total_points', 'previous_points',
       'rank_change', 'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg',
       'last_year_avg_weighted', 'two_year_ago_avg', 'two_year_ago_weighted',
       'three_year_ago_avg', 'three_year_ago_weighted', 'confederation',
       'rank_date'],
      dtype='object')


In [317]:
#dropping the unnecessary columns
## Dropping unnecessary columns

Fifa.drop(columns = ['country_abrv','country', 'total_points', 'country_full_x', 'country_full_y', 'previous_points', 'rank_change', 
                        'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
                        'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg', 
                        'three_year_ago_weighted', 'confederation', 'rank_date'], axis = 1, inplace = True)

In [318]:
#Changing the neutral data from categorical data
venue=pd.get_dummies(Fifa['neutral'],drop_first=True)
venue.head(4)
fifa=pd.concat([Fifa,venue],axis=1)
fifa.head()

,home_team,away_team,home_score,away_score,neutral,home_team_rank,away_team_rank,True
0,Bolivia,Uruguay,3,1,False,59,22,0
1,Bolivia,Uruguay,3,1,False,59,15,0
2,Bolivia,Uruguay,3,1,False,59,17,0
3,Bolivia,Uruguay,3,1,False,59,18,0
4,Bolivia,Uruguay,3,1,False,59,17,0


In [319]:
#import our file now for final data analysis
Fifa.to_csv('fifa.cs')

In [ ]:
#import our new file to start analysis
final=pd.read_csv('fifa.csv')
final.head(5)


Predictive analysis

We will come up with two models that

    Predict number of goals scored by home team
    Predict number of goals scored by away team

In [332]:
# Multicollinearity & VIF checks

corr = final.corr()
print ('Correlation matrix')
print ('....')
print(corr)

print ('VIF matrix')
print ('....')
pd.DataFrame(np.linalg.inv(final.corr().values), index = corr.index, columns=corr.columns)

Correlation matrix
....
            Unnamed: 0  home_score  away_score  ...   neutral  home_rank  away_rank
Unnamed: 0    1.000000    0.152502   -0.104621  ...  0.051951   0.340336   0.549281
home_score    0.152502    1.000000   -0.133919  ...  0.012780  -0.040099   0.306575
away_score   -0.104621   -0.133919    1.000000  ...  0.086808   0.144491  -0.145258
tournament   -0.199256   -0.132261    0.003772  ... -0.109815  -0.263833  -0.323407
neutral       0.051951    0.012780    0.086808  ...  1.000000   0.064096   0.085021
home_rank     0.340336   -0.040099    0.144491  ...  0.064096   1.000000   0.489355
away_rank     0.549281    0.306575   -0.145258  ...  0.085021   0.489355   1.000000

[7 rows x 7 columns]
VIF matrix
....


,Unnamed: 0,home_score,away_score,tournament,neutral,home_rank,away_rank
Unnamed: 0,1.451115,0.000904,0.072636,0.019523,-0.009812,-0.153869,-0.704351
home_score,0.000904,1.172430,0.045421,0.080919,0.011264,0.291032,-0.470543
away_score,0.072636,0.045421,1.105407,0.011498,-0.102340,-0.304184,0.268021
tournament,0.019523,0.080919,0.011498,1.149496,0.091248,0.172000,0.245967
neutral,-0.009812,0.011264,-0.102340,0.091248,1.024929,0.015104,-0.077951
home_rank,-0.153869,0.291032,-0.304184,0.172000,0.015104,1.511749,-0.734331
away_rank,-0.704351,-0.470543,0.268021,0.245967,-0.077951,-0.734331,2.015598


In [333]:
# Predict Home Score for different teams

# Set dependent and independent variables

x = np.array(final[['home_rank', 'away_rank', 'tournament']])
y = final['home_score']

print(x)
print(y)

[[ 59  22   0]
 [ 59  15   0]
 [ 59  17   0]
 ...
 [164 197   0]
 [164 197   0]
 [164 194   0]]
0         3
1         3
2         3
3         3
4         3
         ..
247986    5
247987    5
247988    5
247989    5
247990    5
Name: home_score, Length: 247991, dtype: int64


In [334]:
reg_line_home = LinearRegression()
reg_line_home.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [335]:
#Establish training and testing sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 20)

LINEAR REGRESSION PREDICTION

In [ ]:
lin_reg_home = LinearRegression()
lin_reg_home.fit(x,y)


In [337]:
# Fit linear regression to our polynomial regression model

lin_reg_home = LinearRegression()
lin_reg_home.fit(x,y)
# predict using linear regression
lin_pred_home = reg_line_home.predict

print(lin_pred_home)


<bound method LinearModel.predict of LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)>


POLYNOMIAL REGRESSION

In [338]:
# Fit linear regression to our polynomial regression model

# Fit polynomial regression to the data

poly_reg_home = PolynomialFeatures(degree = 4)
x_poly_home = poly_reg_home.fit_transform(x)
# predict using polynomial regression
poly = lin_reg_home.predict(poly_reg_home.fit_transform([[2]])
print(poly)

SyntaxError: ignored

In [339]:
# Test model accuracy using RMSE

regressor_home = LinearRegression()
regressor_home.fit(x, y)

y_predict_home = regressor_home.predict(x_test)

print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_predict_home)))

Root Mean Squared Error: 1.7016562639253234


K-FOLDS

In [342]:
folds = KFold(n_splits = 5)

# create array to store the RMSE values of each fold
RMSES_home = []

# create a counter
count_home = 1

for train_index, test_index in folds.split(x):
    print("\nTraining model " + str(count_home))
    
    x_train, x_home_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # fit a regression model accordingly
    reg_home = LinearRegression()
    reg_home.fit(x,y)
    
    # assess the accuracy of the model
    y_home_pred = reg_home.predict(x)
    
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_predict_home)),errors='coerce')
RMSES_home.append(rmse_value_home)
    
print('Model ' + str(count_home) + ' Root Mean Squared Error:',rmse_value_home)
count_home += 1


Training model 1

Training model 1

Training model 1

Training model 1

Training model 1


ValueError: ignored

In [ ]:
# set prediction
home_reg = LinearRegression()
home_reg.fit(x_train, y_train)
home_predict = home_reg.predict(x_test)

In [ ]:
# creating residuals
residuals_home = np.subtract(home_predict, y_test)
residuals_home.head()

Predict Away Scores

In [345]:
#Establish predictors and predicted fields

x= np.array(final[['home_rank', 'away_rank','tournament']])
y = fifa['away_score']

print(x)
print(y)

[[ 59  22   0]
 [ 59  15   0]
 [ 59  17   0]
 ...
 [164 197   0]
 [164 197   0]
 [164 194   0]]
0         1
1         1
2         1
3         1
4         1
         ..
247986    0
247987    0
247988    0
247989    0
247990    0
Name: away_score, Length: 247991, dtype: int64


In [346]:
reg_line_home = LinearRegression()
reg_line_home.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [347]:
#Establish training and testing sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 20)

Linear Regression for away team

In [349]:
lin_reg_home = LinearRegression()
lin_reg_home.fit(x,y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Fit linear regression to our polynomial regression model

lin_reg_home = LinearRegression()
lin_reg_home.fit(x,y)
# predict using linear regression
lin_pred_home = reg_line_home.predict([20])

print("Linear Prediction: %d" %lin_pred_home)
print("Linear Prediction (accurate):", lin_pred_home)
print ('-------')

Polynomial Regression

In [353]:
# Fit linear regression to our polynomial regression model

# Fit polynomial regression to the data

poly_reg_home = PolynomialFeatures(degree = 4)
x_poly_home = poly_reg_home.fit_transform(x)
# predict using polynomial regression
poly = lin_reg_home.predict(poly_reg_home.fit_transform([[2]])
print(poly)

SyntaxError: ignored

In [354]:
# Test model accuracy using RMSE

regressor_home = LinearRegression()
regressor_home.fit(x, y)

y_predict_home = regressor_home.predict(x_test)

print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_predict_home)))

Root Mean Squared Error: 1.0962345506432307


K-FOLD

CROSS VALIDATION

In [355]:
folds = KFold(n_splits = 5)

# create array to store the RMSE values of each fold
RMSES_home = []

# create a counter
count_home = 1

for train_index, test_index in folds.split(x):
    print("\nTraining model " + str(count_home))
    
    x_train, x_home_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # fit a regression model accordingly
    reg_home = LinearRegression()
    reg_home.fit(x,y)
    
    # assess the accuracy of the model
    y_home_pred = reg_home.predict(x)
    
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_predict_home)))
RMSES_home.append(rmse_value_home)
    
print('Model ' + str(count_home) + ' Root Mean Squared Error:',rmse_value_home)
count_home += 1


Training model 1

Training model 1

Training model 1

Training model 1

Training model 1


ValueError: ignored

In [356]:
# set prediction
home_reg = LinearRegression()
home_reg.fit(x_train, y_train)
home_predict = home_reg.predict(x_test)

In [357]:
# creating residuals
residuals_home = np.subtract(home_predict, y_test)
residuals_home.head()

ValueError: ignored